# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
K. Lee  ->  K. Lee  |  ['K. Lee']
J. Liu  ->  J. Liu  |  ['J. Liu']
M. Clausen  ->  M. Clausen  |  ['M. Clausen']
Arxiv has 57 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.08387


extracting tarball to tmp_2410.08387...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']


...

Retrieving document from  https://arxiv.org/e-print/2410.08628


extracting tarball to tmp_2410.08628...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.09000


extracting tarball to tmp_2410.09000...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.08387-b31b1b.svg)](https://arxiv.org/abs/2410.08387) | **RUBIES: JWST/NIRSpec resolves evolutionary phases of dusty star-forming galaxies at $z\sim2$**  |
|| O. R. Cooper, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-10-14*|
|*Comments*| *20 pages, 4 figures; submitted to ApJ*|
|**Abstract**|            The dearth of high quality spectroscopy of dusty star-forming galaxies (DSFGs) -- the main drivers of the assembly of dust and stellar mass at the peak of activity in the Universe -- greatly hinders our ability to interpret their physical processes and evolutionary pathways. We present JWST/NIRSpec observations from RUBIES of four submillimeter-selected, ALMA-detected DSFGs at cosmic noon, $z\sim2.3-2.7$. While photometry uniformly suggests vigorous ongoing star formation for the entire sample in line with canonical DSFGs, the spectra differ: one source has spectroscopic evidence of an evolved stellar population, indicating a recent transition to a post-starburst phase, while the remainder show strong spectroscopic signatures of ongoing starbursts. All four galaxies are infrared-luminous (log$_{10}$$L_{\rm{IR}}$/L$_{\rm \odot}$ $>12.4$), massive (log$_{10}\,M_\star$/M$_{\rm \odot}$ $>11$), and very dust-obscured ($A_V\sim3-4$ ABmag). Leveraging detections of multiple Balmer and Paschen lines, we derive an optical attenuation curve consistent with Calzetti overall, yet an optical extinction ratio $R_V\sim2.5$, potentially indicating smaller dust grains or differences in star-dust geometry. This case study provides some of the first detailed spectroscopic evidence that the DSFGs encompass a heterogeneous sample spanning a range of star formation properties and evolutionary stages, and illustrates the advantages of synergistic JWST and ALMA analysis of DSFGs.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.08628-b31b1b.svg)](https://arxiv.org/abs/2410.08628) | **Is the Gum Nebula an Important Interstellar Scattering Disk of Background Pulsars?**  |
|| R. Wang, et al. -- incl., <mark>K. Lee</mark>, <mark>J. Liu</mark> |
|*Appeared on*| *2024-10-14*|
|*Comments*| *Accepted by SCIENCE CHINA Physics, Mechanics & Astronomy*|
|**Abstract**|            The Gum Nebula is a faint supernova remnant extending about 40 degrees across the southern sky, potentially affecting tens of background pulsars. Though the view that the Gum Nebula acts as a potential scattering screen for background pulsars has been recurrently mentioned over the past five decades, it has not been directly confirmed. We chose the strong background pulsar PSR~B0740$-$28 as a probe and monitored its diffractive interstellar scintillation (DISS) at 2.25~$\&$~8.60~GHz simultaneously for about two years using the Shanghai Tian Ma Radio Telescope (TMRT). DISS was detected at both frequencies and quantified by two-dimensional autocorrelation analysis. We calculated their scattering spectral index $\alpha$ and found that 9/21 of the observations followed the theoretical predictions, while 4/21 of them clearly showed $\alpha < 4$. This finding provides strong support for anomalous scattering along the pulsar line of sight, due to the large frequency lever arm and the simultaneous features of our dual-frequency observations. In comparison to the 2.25~GHz observations, scintillation arcs were observed in 10/21 of the secondary spectrum plots for 8.60~GHz observations. Consequently, the highest frequency record for pulsar scintillation arc detection was updated to 8.60~GHz. Our fitting results were the most direct evidence for the view that the Gum Nebula acts as the scattering screen for background pulsars, because both the distance ($245^{+69}_{-72}$~pc) and transverse speed ($22.4^{+4.1}_{-4.2}$~km/s) of the scintillation screen are comparable with related parameters of the Gum Nebula. Our findings indicated that anisotropic scattering provides a superior explanation for the annual modulation of scintillation arcs than isotropic scattering. Additionally, the orientation of its long axis was also fitted.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.09000-b31b1b.svg)](https://arxiv.org/abs/2410.09000) | **The JWST Emission Line Survey (JELS): Extending rest-optical narrow-band emission line selection into the Epoch of Reionization**  |
|| K. J. Duncan, et al. -- incl., <mark>M. Clausen</mark> |
|*Appeared on*| *2024-10-14*|
|*Comments*| *Submitted to MRNAS; 13 pages, 8 figures. Accompanying data will be publicly released upon acceptance of this and companion paper (Pirie et al., subm.)*|
|**Abstract**|            We present the JWST Emission Line Survey (JELS), a Cycle 1 JWST imaging programme exploiting the wavelength coverage and sensitivity of NIRCam to extend narrow-band rest-optical emission line selection into the epoch of reionization (EoR) for the first time, and to enable unique studies of the resolved ionised gas morphology in individual galaxies across cosmic history. The primary JELS observations comprise $\sim4.7\mu$m narrow-band imaging over $\sim63$ arcmin$^{2}$ designed to enable selection of H$\alpha$ emitters at $z\sim6.1$, as well as the selection of a host of novel emission-line samples including [OIII] at $z\sim8.3$ and Pa $\alpha/\beta$ at $z\sim1.5/2.8$. For the prime F466N and F470N narrow-band observations, the emission-line sensitivities achieved are up to $\sim2\times$ more sensitive than current slitless spectroscopy surveys (5$\sigma$ limits of 1.1-1.6$\times10^{-18}\text{erg s}^{-1}\text{cm}^{-2}$), corresponding to unobscured H$\alpha$ star-formation rates (SFRs) of 1-1.6 $\text{M}_{\odot}\,\text{yr}^{-1}$ at $z\sim6.1$ and extending emission-line selections in the EoR to fainter populations. Simultaneously, JELS also obtained F200W broadband and F212N narrow-band imaging (H$\alpha$ at $z\sim2.23$) that probes SFRs $\gtrsim5\times$ fainter than previous ground-based narrow-band studies ($\sim0.2 \text{M}_{\odot}\text{yr}^{-1}$), offering an unprecedented resolved view of star formation at cosmic noon. In this paper we describe the detailed JELS survey design, key data processing steps specific to the survey observations, and demonstrate the exceptional data quality and imaging sensitivity achieved. We then summarise the key scientific goals of JELS and present some early science results, including examples of spectroscopically confirmed H$\alpha$ and [OIII] emitters discovered by JELS that illustrate the novel parameter space probed.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.08563-b31b1b.svg)](https://arxiv.org/abs/2410.08563) | **Simplified model(s) of the GRAVITY+ adaptive optics system(s) for performance prediction**  |
|| A. Berdeu, et al. -- incl., <mark>P. Garcia</mark>, <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-10-14*|
|*Comments*| **|
|**Abstract**|            In the context of the GRAVITY+ upgrade, the adaptive optics (AO) systems of the GRAVITY interferometer are undergoing a major lifting. The current CILAS deformable mirrors (DM, 90 actuators) will be replaced by ALPAO kilo-DMs (43x43, 1432 actuators). On top of the already existing 9x9 Shack-Hartmann wavefront sensors (SH-WFS) for infrared (IR) natural guide star (NGS), new 40x40 SH-WFSs for visible (VIS) NGS will be deployed. Lasers will also be installed on the four units of the Very Large Telescope to provide a laser guide star (LGS) option with 30x30 SH-WFSs and with the choice to either use the 9x9 IR-WFSs or 2x2 VIS-WFSs for low order sensing. Thus, four modes will be available for the GRAVITY+ AO system (GPAO): IR-NGS, IR-LGS, VIS-NGS and VIS-LGS. To prepare the instrument commissioning and help the observers to plan their observations, a tool is needed to predict the performances of the different modes and for different observing conditions (NGS magnitude, science object magnitude, turbulence conditions...) We developed models based on a Mar{é}chal approximation to predict the Strehl ratio of the four GPAO modes in order to feed the already existing tool that simulates the GRAVITY performances. Waiting for commissioning data, our model was validated and calibrated using the TIPTOP toolbox, a Point Spread Function simulator based on the computation of Power Spectrum Densities. In this work, we present our models of the NGS modes of GPAO and their calibration with TIPTOP.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.08387/./absolute_total_attenuationcurve.png', 'tmp_2410.08387/./spec_v3.png', 'tmp_2410.08387/./cmd_almajwst_dustyrubies_as2uds_uds.png']
copying  tmp_2410.08387/./absolute_total_attenuationcurve.png to _build/html/
copying  tmp_2410.08387/./spec_v3.png to _build/html/
copying  tmp_2410.08387/./cmd_almajwst_dustyrubies_as2uds_uds.png to _build/html/
exported in  _build/html/2410.08387.md
    + _build/html/tmp_2410.08387/./absolute_total_attenuationcurve.png
    + _build/html/tmp_2410.08387/./spec_v3.png
    + _build/html/tmp_2410.08387/./cmd_almajwst_dustyrubies_as2uds_uds.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\Lya}{Lyman-\alpha}$
$\newcommand{\Hb}{H\beta}$
$\newcommand{\Pab}{Pa\beta}$
$\newcommand{\Pag}{Pa\gamma}$
$\newcommand{\Pad}{Pa\delta}$
$\newcommand{\Pae}{Pa\epsilon}$
$\newcommand{\Feii}{[Fe\textsc{ii}]\lambda1.257\um}$
$\newcommand{\Ha}{H\alpha}$
$\newcommand{\zs}{z_{\rm spec}}$
$\newcommand{\zp}{z_{\rm phot}}$
$\newcommand{\Hei}{He\textsc{i}}$
$\newcommand{\Oii}{[O\textsc{ii}]}$
$\newcommand{\Hii}{H\textsc{ii}}$
$\newcommand{\Nii}{[N\textsc{ii}]}$
$\newcommand{\Siii}{[S\textsc{iii}]}$
$\newcommand{\Oiii}{[O\textsc{iii}]}$
$\newcommand{\Ciii}{C\textsc{iii}]}$
$\newcommand{\Cii}{[C\textsc{ii}]}$
$\newcommand{\Muv}{M_{\text{UV}}}$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# RUBIES: JWST/NIRSpec resolves evolutionary phases of dusty star-forming galaxies at $z\sim2$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.08387-b31b1b.svg)](https://arxiv.org/abs/2410.08387)<mark>Appeared on: 2024-10-14</mark> -  _20 pages, 4 figures; submitted to ApJ_

</div>
<div id="authors">

O. R. Cooper, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** The dearth of high quality spectroscopy of dusty star-forming galaxies (DSFGs) --- the main drivers of the assembly of dust and stellar mass at the peak of activity in the Universe --- greatly hinders our ability to interpret their physical processes and evolutionary pathways. We present JWST/NIRSpec observations from RUBIES of four submillimeter-selected, ALMA-detected DSFGs at cosmic noon, $z\sim2.3-2.7$ . While photometry uniformly suggests vigorous ongoing star formation for the entire sample in line with canonical DSFGs, the spectra differ: one source has spectroscopic evidence of an evolved stellar population, indicating a recent transition to a post-starburst phase, while the remainder show strong spectroscopic signatures of ongoing starbursts. All four galaxies are infrared-luminous (log $_{10}$ $L_{\rm{IR}}$ / $\Lsun$ $>12.4$ ), massive (log $_{10} M_\star$ / $\Msun$ $>11$ ), and very dust-obscured ( $A_V\sim3-4$ ABmag). Leveraging detections of multiple Balmer and Paschen lines, we derive an optical attenuation curve consistent with Calzetti overall, yet an optical extinction ratio $R_V\sim2.5$ , potentially indicating smaller dust grains or differences in star-dust geometry. This case study provides some of the first detailed spectroscopic evidence that the DSFGs encompass a heterogeneous sample spanning a range of star formation properties and evolutionary stages, and illustrates the advantages of synergistic JWST and ALMA analysis of DSFGs.

</div>

<div id="div_fig1">

<img src="tmp_2410.08387/./absolute_total_attenuationcurve.png" alt="Fig1" width="100%"/>

**Figure 1. -** Dust attenuation curves derived for our sample compared to a few literature attenuation/extinction curves. Attenuation curve measurements and associated errors from line ratios measured for all detected Balmer/Paschen emission line pairs are shown as stars, and the best-fit function linear in $1/\lambda$ is shown in thick purple. Overplotted are the extinction curves for the MW \citep[gray;][]{1989cardelli}, SMC \citep[dark purple;][]{2003gordon}, and LMC \citep[lavender;][]{2003gordon}, as well as attenuation curves measured for star-forming galaxies (SFGs) nearby \citep[dark gold;][]{2000calzetti}, at $z\sim 2$\citep[yellow;][]{2020reddy}, and for a $z=4.41$ galaxy \citep[black;][]{2024sanders}. _Top:_ Absolute attenuation curve normalized to $A_V = 1$, for direct comparison of the relative shapes of the attenuation curves. _Bottom:_ Total attenuation $k^\prime(\lambda)$, to demonstrate the relative magnitudes of attenuation over the wavelength range. Line ratios used for fitting are all anchored to $\Pab$, therefore each attenuation curve here is normalized such that $k^\prime(\rm Pa\beta)=1$. Note that we do not show the $z=4.41$ curve here as it is not constrained out to our normalization wavelength. The legend shows the best fit functional form for $k(\lambda)$; this is the normalized functional form such that $k(28000\rm{Å}) = 0$, which differs in normalization constant only to the curve shown in the figure. (*fig:alaw*)

</div>
<div id="div_fig2">

<img src="tmp_2410.08387/./spec_v3.png" alt="Fig3" width="100%"/>

**Figure 3. -** JWST/NIRSpec 1D PRISM spectra and uncertainty for our sources (green/purple and light green/purple, respectively) and best-fit model from \texttt{msaexp}(black). We present the spectra in two different colors to emphasize the distinction in their star-formation properties, using green for the more evolved, post-starburst galaxy, and purple for the strongly starbursting galaxies. Measured spectroscopic redshifts and object names are listed for each source. Wavelengths of redshifted spectroscopic features are shown as dashed vertical lines, with detected lines or features (S/N$>3$) labeled in dark gray and non-detected lines labeled in light gray for reference. The JWST/NIRCam F444W cutout for each source is shown as an inset ($2"$ wide), with the position of the NIRSpec slit overplotted on each image. (*fig:spec*)

</div>
<div id="div_fig3">

<img src="tmp_2410.08387/./cmd_almajwst_dustyrubies_as2uds_uds.png" alt="Fig2" width="100%"/>

**Figure 2. -** Our DSFGs are bright and red, have typical NIRCam colors and moderately bright submillimeter flux densities, overall consistent with the larger DSFG sample. _Top:_ JWST/NIRCam color magnitude diagram comparing the F277W $-$ F444W colors and F444W magnitudes of the DSFGs reported in this work to the broader sample of known DSFGs in AS2UDS \citep[][]{2020dudz,2024gillman} within the PRIMER-UDS footprint (black points), and to the DSFGs in SCUBADive \citep[gray hexagons;][]{2024mckinney}, with the full PRIMER-UDS catalog in the background (small gray points). The three starbursting galaxies in our sample are noted in purple, while the post-starburst galaxy is shown in green. _Bottom:_ Color magnitude diagram with JWST/NIRCam F150W $-$ F444W color versus ALMA Band 7 flux density. Symbols are consistent as in the top subplot, but here colors denote the redshift; a spectroscopic redshift for the four RUBIES DSFGs, else the best-available redshifts from AS2UDS and SCUBADive. (*fig:cmd*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.08387"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

216  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
